In [1]:
import pandas as pd
from tqdm.notebook import tqdm
from multiprocessing import Process, Manager, Pool

## load trabalhadores

In [2]:
df_trabalhadores = pd.read_csv("data/D_ETL_IMO_EXTRACAO_SINE_ABERTO_TRABALHADORES_SP.csv", sep=";", encoding="iso8859-1")

/home/chris/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df_trabalhadores.dropna(subset=['PRETENSOES'], inplace=True)
df_trabalhadores[['PRETENSOES','LIXO']] = df_trabalhadores.PRETENSOES.str.split("(",1,expand=True)

In [4]:
df_trabalhadores = df_trabalhadores[:30000]
df_trabalhadores['id_trabalhador'] = df_trabalhadores.index
print(df_trabalhadores.shape)
df_trabalhadores.head()

(30000, 22)


,NACIONALIDADE,DEFICIENCIAS,BAIRRO,CEP,CODIGO_MUNICIPIO_IBGE,NOME_MUNICIPIO,UF,ESCOLARIDADE,ESTUDANTE,CURSOS_PROFISSIONALIZANTES,...,IDIOMAS,HABILITACAO,VEICULOS,DISP_VIAJAR,DISP_DORMIR_EMP,DISP_AUSENTAR_DOMIC,PRETENSOES,MUNICIPIOS_INTERESSE,LIXO,id_trabalhador
0,BRASILEIRA,NaN,VILA UNIDOS,1.22145e+07,354990,SAO JOSE DOS CAMPOS,SP,Superior Incompleto,S,NaN,...,NaN,AB,N,N,N,N,521110-Vendedor de comércio varejista,NaN,"N,54,0,Indiferente)",0
1,BRASILEIRA,NaN,MONTANHAO,9.78414e+06,354870,SAO BERNARDO DO CAMPO,SP,Médio Completo,N,NaN,...,NaN,Nenhum,N,N,N,N,513505-Ajudante de cozinha,NaN,"N,0,0,Indiferente)|514320-Auxiliar de limpeza(...",1
2,BRASILEIRA,NaN,JARDIM SANTA TEREZINHA (ZON,8.43002e+06,355030,SAO PAULO,SP,Médio Completo,N,NaN,...,NaN,Nenhum,N,N,N,N,411005-Auxiliar administrativo,NaN,"N,0,0,Indiferente)|413110-Auxiliar financeiro(...",2
3,BRASILEIRA,NaN,NaN,6.62613e+06,352500,JANDIRA,SP,Médio Completo,S,NaN,...,NaN,Nenhum,N,S,N,N,413205-ATENDENTE DE AGENCIA,NaN,"S,0,0,Indiferente)|422305-OPERADOR DE TELEMARK...",3
4,BRASILEIRA,NaN,NaN,1.80654e+07,355220,SOROCABA,SP,Médio Completo,N,NaN,...,NaN,Nenhum,N,S,S,S,422305-OPERADOR DE TELEMARKETING ATIVO,NaN,"N,17,0,Indiferente)|514320-Auxiliar de limpeza...",4


## load vagas

In [5]:
df_vagas = pd.read_csv("data/vagas_mock.csv")
df_vagas = df_vagas[:10000]
print(df_vagas.shape)
df_vagas.head()

(10000, 7)


,cidade,escolaridade,graduacao,id_empresa,industria,nome_empresa,posicao
0,PIRACICABA,Médio Completo,Nenhum,cd717d64-1c29-4bfe-b074-005ab8034621,Produção,enhance B2B mindshare,641015-Tratorista agrícola
1,CAIEIRAS,Fundamental Completo,Nenhum,f71c2bd0-6f3f-4bd0-83b6-8402d65830c5,Petróleo,implement killer e-commerce,414105-Auxiliar de almoxarifado
2,PEDERNEIRAS,Fundamental Completo,Nenhum,d7034759-ba70-4f97-8bf9-7fd503339852,Fumo,e-enable cutting-edge e-markets,421105-ATENDENTE COMERCIAL
3,VALINHOS,Médio Completo,Nenhum,86ef07ac-4e84-4479-aa89-1d111a99240e,Financeiro,reinvent real-time convergence,411005-Auxiliares administrativos e de escritó...
4,DIADEMA,Médio Completo,Nenhum,8faea19c-6d81-49f6-8148-fb96e0b3c473,Comércio,benchmark killer networks,782305-Motorista de carro de passeio


## Feature engineering
automaticamente "avalia" uma posicao para um trabalhador

In [7]:
job_list = []
for index, row in tqdm(df_trabalhadores[:10].iterrows(), total=df_trabalhadores.shape[0]):
    print(row['PRETENSOES'])
    for index_vaga, row_vaga in df_vagas.iterrows():
        joblist.append()

521110-Vendedor de comércio varejista



NameError: name 'joblist' is not defined

In [ ]:
def get_rating(shared_list, id_trabalhador, escolaridade_trabalhador, graduacao_trabalhador, 
               cidade_trabalhador, escolaridade_vaga, graduacao_vaga, cidade_vaga, id_vaga):
    points = 0
    if escolaridade_trabalhador == escolaridade_vaga:
        points += 1
    if graduacao_trabalhador == graduacao_vaga:
        points += 1
    if cidade_trabalhador == cidade_vaga:
        points += 1
        
    shared_list.append({'id_trabalhador': id_trabalhador, 'id_vaga':id_vaga, 'rating':rating})